In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName('rheeza').getOrCreate()

C:\Users\MODUPE\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.7_qbz5n2kfra8p0\LocalCache\local-packages\Python37\site-packages\pyspark\context.py:317: FutureWarning: Python 3.7 support is deprecated in Spark 3.4.
  warnings.warn("Python 3.7 support is deprecated in Spark 3.4.", FutureWarning)


Understanding the Dataset


In [2]:
##reading the dataset with spark
trials_df = spark.read.json('dataset.json', multiLine=True)

In [3]:
trials_df.show(5)

+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|ageofparticipant|           clinician|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|              result|
+----------------+--------------------+---------+-----------------+-------------------+------------------------------+--------------------+
|              19|{Ontario, Saul, t...|  Placebo|    1619827200000|      1617235200000|                            52|{BP normalized, r...|
|              14|{Ontario, Saul, n...| Naproxen|    1619827200000|      1617235200000|                            78|    {Follow-up, N/A}|
|              17|{Ontario, Saul, n...|  Placebo|    1619827200000|      1617235200000|                            14|    {Follow-up, N/A}|
|              18|{Ontario, Will, n...| Naproxen|    1619827200000|      1617235200000|                            14|{BP normalized, N/A}|
|              17|{O

In [5]:
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [6]:
trials_df.columns

['ageofparticipant',
 'clinician',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result']

In [7]:
trials_df.dtypes

[('ageofparticipant', 'bigint'),
 ('clinician', 'struct<branch:string,name:string,role:string>'),
 ('drug_used', 'string'),
 ('experimentenddate', 'string'),
 ('experimentstartdate', 'string'),
 ('noofhourspassedatfirstreaction', 'bigint'),
 ('result', 'struct<conclusion:string,sideeffectsonparticipant:string>')]

In [9]:
## data exploration
trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- clinician: struct (nullable = true)
 |    |-- branch: string (nullable = true)
 |    |-- name: string (nullable = true)
 |    |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- result: struct (nullable = true)
 |    |-- conclusion: string (nullable = true)
 |    |-- sideeffectsonparticipant: string (nullable = true)



In [8]:
columns = ['ageofparticipant',
 'clinician.branch',
 'clinician.name',
 'clinician.role',
 'drug_used',
 'experimentenddate',
 'experimentstartdate',
 'noofhourspassedatfirstreaction',
 'result.conclusion',
 'result.sideeffectsonparticipant'
 ]

In [9]:
## create a copy of the dataset and also flatten the columns
new_trials_df = trials_df.select(columns)
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [13]:
new_trials_df.printSchema()

root
 |-- ageofparticipant: long (nullable = true)
 |-- branch: string (nullable = true)
 |-- name: string (nullable = true)
 |-- role: string (nullable = true)
 |-- drug_used: string (nullable = true)
 |-- experimentenddate: string (nullable = true)
 |-- experimentstartdate: string (nullable = true)
 |-- noofhourspassedatfirstreaction: long (nullable = true)
 |-- conclusion: string (nullable = true)
 |-- sideeffectsonparticipant: string (nullable = true)



In [10]:
new_trials_df.show(5)

+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|ageofparticipant| branch|   name|     role|drug_used|experimentenddate|experimentstartdate|noofhourspassedatfirstreaction|   conclusion|sideeffectsonparticipant|
+----------------+-------+-------+---------+---------+-----------------+-------------------+------------------------------+-------------+------------------------+
|              19|Ontario|   Saul|therapist|  Placebo|    1619827200000|      1617235200000|                            52|BP normalized|          rashes on neck|
|              14|Ontario|   Saul|    nurse| Naproxen|    1619827200000|      1617235200000|                            78|    Follow-up|                     N/A|
|              17|Ontario|   Saul|    nurse|  Placebo|    1619827200000|      1617235200000|                            14|    Follow-up|                     N/A|
|              18|Onta

In [11]:
import pyspark.sql.functions as fn


In [12]:
trials_df.select([fn.count(fn.when(fn.col(column).isNull(), column)).alias()for column in columns]).show

Py4JJavaError: An error occurred while calling o29.select.
: org.apache.spark.SparkException: [INTERNAL_ERROR] Found the unresolved operator: 'Aggregate [count(CASE WHEN isnull(ageofparticipant#0L) THEN ageofparticipant END) AS (), count(CASE WHEN isnull(clinician#1.branch) THEN clinician.branch END) AS (), count(CASE WHEN isnull(clinician#1.name) THEN clinician.name END) AS (), count(CASE WHEN isnull(clinician#1.role) THEN clinician.role END) AS (), count(CASE WHEN isnull(drug_used#2) THEN drug_used END) AS (), count(CASE WHEN isnull(experimentenddate#3) THEN experimentenddate END) AS (), count(CASE WHEN isnull(experimentstartdate#4) THEN experimentstartdate END) AS (), count(CASE WHEN isnull(noofhourspassedatfirstreaction#5L) THEN noofhourspassedatfirstreaction END) AS (), count(CASE WHEN isnull(result#6.conclusion) THEN result.conclusion END) AS (), count(CASE WHEN isnull(result#6.sideeffectsonparticipant) THEN result.sideeffectsonparticipant END) AS ()]
	at org.apache.spark.SparkE

In [18]:
import pyspark
print(pyspark.__version__)

import py4j
print(py4j.__version__)

3.4.1
0.10.9.7


In [13]:
## rename columns
new_column_name = {
 'ageofparticipant': 'age_of_participant'
 ,'branch': 'clinic_branch'
 , 'name' : 'clinician_head'
 , 'role' : 'assistants_role'
 , 'experimentenddate' : 'experiment_end_date'
 , 'experimentstartdate' : 'experiment_start_date'
 , 'noofhourspassedatfirstreaction' : 'hours_passed_at_first_reaction'
 , 'sideeffectsonparticipant' : 'observed_side_effect'  
}

new_trials_df = new_trials_df.withColumnsRenamed(new_column_name)
new_trials_df.show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|age_of_participant|clinic_branch|clinician_head|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|
+------------------+-------------+--------------+---------------+---------+-----

In [14]:
new_trials_df.dtypes

[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('clinician_head', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('observed_side_effect', 'string')]

In [23]:
new_trials_df.describe().show(2)

+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+----------+--------------------+
|summary|age_of_participant|clinic_branch|clinician_head|assistants_role|drug_used| experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|conclusion|observed_side_effect|
+-------+------------------+-------------+--------------+---------------+---------+--------------------+---------------------+------------------------------+----------+--------------------+
|  count|              3586|         3586|          3586|           3586|     3586|                3586|                 3586|                          3513|      3586|                3586|
|   mean|17.507250418293363|         null|          null|           null|     null|1.618381578137200...| 1.615813671834913...|             44.89097637346997|      null|                null|
+-------+------------------+-------------+--------

In [22]:
new_trials_df = new_trials_df.na.fill({'conclusion': 'unknown', 'assistants_role' : 'unknown', }) ## fill null values with unknown

TRANSFORMATION


In [ ]:
## change the exp end and start date datatype

In [24]:
new_trials_df.dtypes


[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('clinician_head', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('observed_side_effect', 'string')]

In [26]:
from pyspark.sql import types as dtypes

In [29]:
# convert the datatypes to int (long)
# divide by 1000
# convert from unix to normal datetime


new_trials_df.withColumn('start_timestamp', fn.col('experiment_start_date').cast(dtypes.LongType())/1000).show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+---------------+
|age_of_participant|clinic_branch|clinician_head|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|start_timestamp|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+---------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|    1.6172352E9|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|    1.6172352E9|


In [30]:
new_trials_df.withColumn('start_timestamp', fn.from_unixtime(fn.col('experiment_start_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss') ).show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-------------------+
|age_of_participant|clinic_branch|clinician_head|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|    start_timestamp|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|2021-04-01 01:00:00|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1617235200000|                            78|    Follow-up|                 N/A|2

In [31]:
from pyspark.sql.functions import to_timestamp

In [32]:


new_trials_df = new_trials_df\
    .withColumn('start_timestamp', fn.from_unixtime(fn.col('experiment_start_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
    .withColumn('start_timestamp', fn.col('start_timestamp').cast(dtypes.TimestampType()))\
            .withColumn('end_timestamp', fn.from_unixtime(fn.col('experiment_end_date').cast(dtypes.LongType())/1000, 'yyyy-MM-dd HH:mm:ss'))\
            .withColumn('end_timestamp', fn.col('end_timestamp').cast(dtypes.TimestampType()))

In [33]:
new_trials_df.dtypes

[('age_of_participant', 'bigint'),
 ('clinic_branch', 'string'),
 ('clinician_head', 'string'),
 ('assistants_role', 'string'),
 ('drug_used', 'string'),
 ('experiment_end_date', 'string'),
 ('experiment_start_date', 'string'),
 ('hours_passed_at_first_reaction', 'bigint'),
 ('conclusion', 'string'),
 ('observed_side_effect', 'string'),
 ('start_timestamp', 'timestamp'),
 ('end_timestamp', 'timestamp')]

In [34]:
new_trials_df.show(2)

+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-------------------+-------------------+
|age_of_participant|clinic_branch|clinician_head|assistants_role|drug_used|experiment_end_date|experiment_start_date|hours_passed_at_first_reaction|   conclusion|observed_side_effect|    start_timestamp|      end_timestamp|
+------------------+-------------+--------------+---------------+---------+-------------------+---------------------+------------------------------+-------------+--------------------+-------------------+-------------------+
|                19|      Ontario|          Saul|      therapist|  Placebo|      1619827200000|        1617235200000|                            52|BP normalized|      rashes on neck|2021-04-01 01:00:00|2021-05-01 01:00:00|
|                14|      Ontario|          Saul|          nurse| Naproxen|      1619827200000|        1